In [ ]:
!pip install scipy seaborn matplotlib cuml-cu11 pyyaml

# Imports

In [1]:
import os
import pickle
import re
import shutil
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from itertools import product

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml
from matplotlib.backends.backend_pdf import PdfPages
from tools import load_npy, load_yaml_as_df, load_pkl, exist_metric, exist_stf_metric, inverse_stf_metrics, keep_split, is_full_group, load_metric_from_log

plt.style.use('default')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rc('font', family='Arial')
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.size'] = 10

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Baselines local

## load data

In [10]:
root = '/data/home/Licheng/workspace/ProjDF-Meta/results_ML3/baselines'
exp_dirs = os.listdir(root)
exp_dirs = [os.path.join(root, exp_dir) for exp_dir in exp_dirs]

params = ['model', 'pred_len', 'data_id', 'learning_rate', 'batch_size', 'patience', 'individual', 'train_epochs', 'lradj']
metric_names = ['mse', 'mae']

df = []
for exp_dir in exp_dirs:
    runned, setting_dir = exist_metric(exp_dir)
    if not runned:
        continue

    config = load_yaml_as_df(os.path.join(setting_dir, 'config.yaml'))
    metric = load_npy(os.path.join(setting_dir, 'metrics.npy'))
    result = config[params]
    model, individual = config['model'].values[0], config['individual'].values[0]
    if model == 'DLinear' and individual:
        result['model'].values[0] = 'DLinear_Ind'
    result.loc[:, metric_names] = metric[1], metric[0]
    result.loc[:, ['exp_dir']] = exp_dir
    df.append(result)

df = pd.concat(df, ignore_index=True)
df.sort_values(by=['model', 'data_id', 'pred_len'], inplace=True)

# save_root = '/data/home/Licheng/workspace/TSF-CCA/stats_CCA'
# os.makedirs(save_root, exist_ok=True)
# df.to_csv(f"{save_root}/baseline_local.csv", index=False)

df.head(4)

FileNotFoundError: [Errno 2] No such file or directory: '/data/home/Licheng/workspace/ProjDF-Meta/results_ML3/baselines'

## pre-load

In [11]:
save_root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/logs'
df = pd.read_csv(f"{save_root}/baselines_chosen.csv")
df.head()

,model,pred_len,data_id,learning_rate,batch_size,patience,individual,train_epochs,lradj,mse,mae,exp_dir
0,Autoformer,96,ECL,0.0005,32,3,0,10,type1,0.188511,0.303782,/data/home/Licheng/workspace/TSF-CCA/results_C...
1,Autoformer,192,ECL,0.0005,32,3,0,10,type1,0.270894,0.370508,/data/home/Licheng/workspace/TSF-CCA/results_C...
2,Autoformer,336,ECL,0.0005,32,3,0,10,type1,0.242762,0.352282,/data/home/Licheng/workspace/TSF-CCA/results_C...
3,Autoformer,720,ECL,0.0005,32,3,0,10,type1,0.294784,0.388354,/data/home/Licheng/workspace/TSF-CCA/results_C...
4,Autoformer,96,ETTh1,0.0005,32,3,0,10,type1,0.448748,0.464985,/data/home/Licheng/workspace/TSF-CCA/results_C...


## analysis

In [6]:
min_mode = 'each'

df2 = df.copy()

columns = ['model', 'data_id', 'learning_rate', 'batch_size', 'patience', 'individual', 'train_epochs']
if min_mode == 'group':
    mse_mean = df2.groupby(columns)['mse'].mean().reset_index()
    idx = mse_mean.groupby(['model', 'data_id'])['mse'].idxmin()
    best_model = mse_mean.loc[idx]
    df2 = df2.merge(best_model[columns], on=columns, how='inner')
elif min_mode == 'each':
    min_mse_idx = df2.groupby(['model', 'data_id', 'pred_len'])['mse'].idxmin()
    df2 = df2.loc[min_mse_idx]

df2 = df2[['model', 'pred_len', 'data_id', 'mse', 'mae']]

dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Traffic', 'Weather', 'PEMS03', 'PEMS08']
df2['data_id'] = pd.Categorical(df2['data_id'], categories=dst_order, ordered=True)

model_order = ['TQNet', 'PDF', 'CycleNet', 'PatchTST', 'DLinear', 'Fredformer', 'iTransformer', 'FreTS', 'TimesNet', 'MICN', 'TiDE']
df2 = df2[df2['model'].isin(model_order)]
df2['model'] = pd.Categorical(df2['model'], categories=model_order, ordered=True)

df2_avg = df2.groupby(['model', 'data_id']).mean(numeric_only=True).reset_index()
df2_avg['pred_len'] = 'Avg'
df2 = pd.concat([df2, df2_avg]).reset_index(drop=True)

df2.sort_values(by=['model', 'data_id', 'pred_len'], inplace=True)

df2 = df2.set_index(['data_id', 'pred_len', 'model']).unstack('model').swaplevel(axis=1)
columns = []
for model in df2.columns.levels[0]:
    columns.append((model, 'mse'))
    columns.append((model, 'mae'))
df2 = df2[columns]

# df2.to_excel(f'{save_root}/baselines_local.xlsx')
# df2.to_csv(f'{save_root}/baselines_local.csv')
df2

/tmp/ipykernel_253782/2260451254.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df2_avg = df2.groupby(['model', 'data_id']).mean(numeric_only=True).reset_index()


model                TQNet                 PDF           CycleNet      \
                       mse       mae       mse       mae      mse mae   
data_id pred_len                                                        
ETTm1   96        0.306602  0.348971  0.320094  0.358292      NaN NaN   
        192       0.352414  0.376268  0.360833  0.379950      NaN NaN   
        336       0.382595  0.397512  0.390477  0.400542      NaN NaN   
        720       0.441163  0.434478  0.451229  0.436571      NaN NaN   
        Avg       0.370693  0.389307  0.380658  0.393839      NaN NaN   
ETTm2   96        0.170291  0.252600  0.174904  0.262821      NaN NaN   
        192       0.233855  0.293836  0.242344  0.305705      NaN NaN   
        336       0.290130  0.331435  0.302503  0.343900      NaN NaN   
        720       0.386822  0.388532  0.401535  0.399057      NaN NaN   
        Avg       0.270274  0.316601  0.280322  0.327871      NaN NaN   
ETTh1   96        0.365111  0.388897  0.374883  0.391447      NaN NaN   
        192       0.427475  0.421499  0.423170  0.419380      NaN NaN   
        336       0.465918  0.448515  0.461447  0.439141      NaN NaN   
        720       0.464051  0.460899  0.483943  0.468018      NaN NaN   
        Avg       0.430639  0.429952  0.435861  0.429496      NaN NaN   
ETTh2   96        0.285516  0.337423  0.287638  0.337379      NaN NaN   
        192       0.361035  0.388219  0.371993  0.389024      NaN NaN   
        336       0.407637  0.422494  0.411470  0.423844      NaN NaN   
        720       0.417388  0.437461  0.418308  0.437993      NaN NaN   
        Avg       0.367894  0.396399  0.372352  0.397060      NaN NaN   
ECL     96        0.134702  0.228727  0.172002  0.255119      NaN NaN   
        192       0.153000  0.244973  0.176853  0.261003      NaN NaN   
        336       0.169068  0.262404  0.192442  0.277235      NaN NaN   
        720       0.202346  0.291418  0.234418  0.312323      NaN NaN   
        Avg       0.164779  0.256880  0.193929  0.276420      NaN NaN   
Traffic 96        0.403745  0.262530  0.451431  0.290025      NaN NaN   
        192       0.424102  0.272680  0.461381  0.293929      NaN NaN   
        336       0.439090  0.277409  0.475661  0.299798      NaN NaN   
        720       0.479501  0.296024       NaN       NaN      NaN NaN   
        Avg       0.436610  0.277161  0.462824  0.294584      NaN NaN   
Weather 96        0.158300  0.200693  0.175676  0.218337      NaN NaN   
        192       0.206637  0.244936  0.225431  0.259884      NaN NaN   
        336       0.262811  0.286290  0.279804  0.298935      NaN NaN   
        720       0.342250  0.339154  0.356894  0.347133      NaN NaN   
        Avg       0.242499  0.267768  0.259451  0.281072      NaN NaN   
PEMS03  12        0.064309  0.167092       NaN       NaN      NaN NaN   
        24        0.080374  0.188680       NaN       NaN      NaN NaN   
        36        0.097987  0.208369       NaN       NaN      NaN NaN   
        48        0.111981  0.223476       NaN       NaN      NaN NaN   
        Avg       0.088663  0.196904       NaN       NaN      NaN NaN   
PEMS08  12        0.074487  0.175849       NaN       NaN      NaN NaN   
        24        0.104101  0.207774       NaN       NaN      NaN NaN   
        36        0.134275  0.236954       NaN       NaN      NaN NaN   
        48        0.168359  0.263294       NaN       NaN      NaN NaN   
        Avg       0.120305  0.220968       NaN       NaN      NaN NaN   

model            PatchTST     DLinear     Fredformer           iTransformer  \
                      mse mae     mse mae        mse       mae          mse   
data_id pred_len                                                              
ETTm1   96            NaN NaN     NaN NaN   0.312016  0.352230     0.322717   
        192           NaN NaN     NaN NaN   0.358952  0.379518     0.384773   
        336           NaN NaN     NaN NaN   0.391882  0.403642     0.415041   
        720          

# Finetune

## load data

In [4]:
root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/results_ML3/finetune'
exp_dirs = os.listdir(root)
exp_dirs = [os.path.join(root, exp_dir) for exp_dir in exp_dirs]

params = ['model', 'pred_len', 'data_id', 'learning_rate', 'inner_lr', 'meta_lr', 'rec_lambda', 'auxi_lambda', 'reg_lambda', 'lradj', 'train_epochs', 'patience', 'batch_size', 'auxi_batch_size', 'warmup_steps', 'meta_inner_steps', 'overlap_ratio', 'num_tasks', 'max_norm', 'auxi_loss', 'first_order', 'dropout', 'cycle']
metric_names = ['mse', 'mae', 'cov']

df = []
for exp_dir in exp_dirs:
    if 'MAE' in exp_dir:
        continue
    # if 'Traffic' not in exp_dir and 'ECL' not in exp_dir and 'Weather' not in exp_dir:
    #     continue
    # if 'Traffic' not in exp_dir or 'TQNet' not in exp_dir:
    #     continue
    runned, setting_dir = exist_metric(exp_dir)
    if not runned:
        continue

    config = load_yaml_as_df(os.path.join(setting_dir, 'config.yaml'))
    metric = load_npy(os.path.join(setting_dir, 'metrics.npy'))
    result = config[params]
    if len(metric) == 6:
        log_metrics = load_metric_from_log(os.path.join(exp_dir, 'result_long_term_forecast.txt'))
        cov_loss = log_metrics['cov']
        result.loc[:, metric_names] = metric[1], metric[0], cov_loss
    else:
        result.loc[:, metric_names] = metric[1], metric[0], metric[2]
    result.loc[:, ['meta_type']] = config[['meta_type']] if 'meta_type' in config.columns else 'all'
    result.loc[:, ['exp_dir']] = exp_dir
    df.append(result)

df = pd.concat(df, ignore_index=True)

df.sort_values(by=['model', 'data_id', 'pred_len'], inplace=True)

save_root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3'
os.makedirs(save_root, exist_ok=True)
df.to_csv(f"{save_root}/finetune_all_results.csv", index=False)

df.head(4)

,model,pred_len,data_id,learning_rate,inner_lr,meta_lr,rec_lambda,auxi_lambda,reg_lambda,lradj,train_epochs,patience,batch_size,auxi_batch_size,warmup_steps,meta_inner_steps,overlap_ratio,num_tasks,max_norm,auxi_loss,first_order,dropout,cycle,mse,mae,cov,meta_type,exp_dir
36,Fredformer,96,ECL,0.002,0.002,0.05,1.0,0.0,0.0,type1,100,5,32,64,300,1,0.0,3,5.0,MSE,1,0.2,24,0.154554,0.245372,0.103599,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
37,Fredformer,96,ECL,0.002,0.002,0.05,1.0,0.0,0.0,type1,100,5,32,64,300,2,0.0,3,5.0,MSE,1,0.2,24,0.153540,0.244694,0.102987,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
38,Fredformer,96,ECL,0.002,0.002,0.10,1.0,0.0,0.0,type1,100,5,32,64,300,1,0.0,3,5.0,MSE,1,0.2,24,0.152893,0.244035,0.087282,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
39,Fredformer,96,ECL,0.002,0.002,0.10,1.0,0.0,0.0,type1,100,5,32,64,300,2,0.0,3,5.0,MSE,1,0.2,24,0.153492,0.244443,0.087609,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...


## pre-load

In [2]:
save_root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3'
df = pd.read_csv(f'{save_root}/finetune_all_results.csv')

df.head(4)

,model,pred_len,data_id,learning_rate,inner_lr,meta_lr,rec_lambda,auxi_lambda,reg_lambda,lradj,train_epochs,patience,batch_size,auxi_batch_size,warmup_steps,meta_inner_steps,overlap_ratio,num_tasks,max_norm,auxi_loss,first_order,dropout,cycle,mse,mae,cov,meta_type,exp_dir
0,Fredformer,96,ECL,0.002,0.002,0.05,1.0,0.0,0.0,type1,100,5,32,64,300,1,0.0,3,5.0,MSE,1,0.2,24,0.154554,0.245372,0.103599,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
1,Fredformer,96,ECL,0.002,0.002,0.05,1.0,0.0,0.0,type1,100,5,32,64,300,2,0.0,3,5.0,MSE,1,0.2,24,0.153540,0.244694,0.102987,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
2,Fredformer,96,ECL,0.002,0.002,0.10,1.0,0.0,0.0,type1,100,5,32,64,300,1,0.0,3,5.0,MSE,1,0.2,24,0.152893,0.244035,0.087282,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
3,Fredformer,96,ECL,0.002,0.002,0.10,1.0,0.0,0.0,type1,100,5,32,64,300,2,0.0,3,5.0,MSE,1,0.2,24,0.153492,0.244443,0.087609,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...


## pre-analysis

In [33]:
df_sort = df.sort_values(by=['model', 'data_id', 'pred_len', 'mse'])[['data_id', 'pred_len', 'mse']].groupby(['data_id', 'pred_len']).head(5)

columns = ['model', 'pred_len', 'data_id', 'mse', 'mae', 'cov', 'learning_rate', 'inner_lr', 'meta_lr', 'auxi_loss', 'rec_lambda', 'auxi_lambda', 'reg_lambda', 'lradj', 'train_epochs', 'patience', 'batch_size', 'auxi_batch_size', 'warmup_steps', 'meta_inner_steps', 'overlap_ratio', 'num_tasks', 'max_norm', 'first_order', 'dropout', 'cycle']

df_sort = df.sort_values(by=['model', 'data_id', 'pred_len', 'mse']).groupby(['data_id', 'num_tasks', 'meta_inner_steps', 'pred_len']).head(5)
df_sort = df_sort[df_sort['data_id'].isin(['Traffic'])]
df_sort[columns]

for i, row in df_sort.iterrows():
    # exp_dir = row['exp_dir']
    mse = row['mse']
    print(f"Data ID: {row['data_id']}, Pred Len: {row['pred_len']}, MSE: {mse:.7f}, inner steps: {row['meta_inner_steps']}, ntask: {row['num_tasks']}, lr: {row['learning_rate']}, meta_lr: {row['meta_lr']}, bs: {row['batch_size']}, auxi_bs: {row['auxi_batch_size']}, warmup: {row['warmup_steps']}")
    # print(exp_dir)

Data ID: Traffic, Pred Len: 96, MSE: 0.4160268, inner steps: 2, ntask: 4, lr: 0.005, meta_lr: 0.1, bs: 8, auxi_bs: 32, warmup: 200
Data ID: Traffic, Pred Len: 96, MSE: 0.4163281, inner steps: 1, ntask: 3, lr: 0.005, meta_lr: 0.15, bs: 16, auxi_bs: 64, warmup: 100
Data ID: Traffic, Pred Len: 96, MSE: 0.4163281, inner steps: 1, ntask: 3, lr: 0.005, meta_lr: 0.15, bs: 16, auxi_bs: 64, warmup: 100
Data ID: Traffic, Pred Len: 96, MSE: 0.4166446, inner steps: 1, ntask: 3, lr: 0.005, meta_lr: 0.2, bs: 16, auxi_bs: 64, warmup: 300
Data ID: Traffic, Pred Len: 96, MSE: 0.4168713, inner steps: 2, ntask: 4, lr: 0.004, meta_lr: 0.01, bs: 8, auxi_bs: 32, warmup: 200
Data ID: Traffic, Pred Len: 96, MSE: 0.4169695, inner steps: 2, ntask: 3, lr: 0.004, meta_lr: 0.03, bs: 16, auxi_bs: 64, warmup: 300
Data ID: Traffic, Pred Len: 96, MSE: 0.4169772, inner steps: 1, ntask: 3, lr: 0.005, meta_lr: 0.1, bs: 16, auxi_bs: 64, warmup: 200
Data ID: Traffic, Pred Len: 96, MSE: 0.4169828, inner steps: 2, ntask: 3, 

## analysis

In [3]:
min_mode = 'each'

df2 = df.copy()
df2 = df2[df2.auxi_loss == 'MSE']
df2 = df2[df2.meta_type == 'all']
df2 = df2[df2.first_order == 1]

columns = ['model', 'data_id', 'learning_rate', 'alpha', 'rank_ratio', 'align_type', 'individual']
if min_mode == 'group':
    mse_mean = df2.groupby(columns)['mse'].mean().reset_index()
    idx = mse_mean.groupby(['model', 'data_id'])['mse'].idxmin()
    best_model = mse_mean.loc[idx]
    df2 = df2.merge(best_model[columns], on=columns, how='inner')
elif min_mode == 'each':
    min_mse_idx = df2.groupby(['model', 'data_id', 'pred_len'])['mse'].idxmin()
    df2 = df2.loc[min_mse_idx]
elif min_mode == 'sum':
    # 新增：mse+mae最小
    df2['sum_error'] = df2['mse'] + df2['mae']
    min_sum_idx = df2.groupby(['model', 'data_id', 'pred_len'])['sum_error'].idxmin()
    df2 = df2.loc[min_sum_idx]
    df2 = df2.drop(columns=['sum_error'])

columns = ['model', 'pred_len', 'data_id', 'mse', 'mae', 'cov', 'learning_rate', 'inner_lr', 'meta_lr', 'auxi_loss', 'rec_lambda', 'auxi_lambda', 'reg_lambda', 'lradj', 'train_epochs', 'patience', 'batch_size', 'auxi_batch_size', 'warmup_steps', 'meta_inner_steps', 'overlap_ratio', 'num_tasks', 'max_norm', 'first_order', 'dropout', 'cycle', 'meta_type', 'exp_dir']
df2 = df2[columns]

dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Traffic', 'Weather', 'PEMS03', 'PEMS08']
df2['data_id'] = pd.Categorical(df2['data_id'], categories=dst_order, ordered=True)

model_order = ['TQNet', 'PDF', 'Fredformer', 'iTransformer']
df2['model'] = pd.Categorical(df2['model'], categories=model_order, ordered=True)
df2.sort_values(by=['model', 'data_id', 'pred_len'], inplace=True)

save_root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3'
os.makedirs(save_root, exist_ok=True)
df2.to_csv(f"{save_root}/finetune_best.csv", index=False)

# print(df2)
df2_avg = df2.groupby(['model', 'data_id']).mean(numeric_only=True).reset_index()
df2_avg['pred_len'] = 'Avg'

df2 = pd.concat([df2, df2_avg]).reset_index(drop=True)

# pl_order = [96, 192, 336, 720, 'Avg']
# df2['pred_len'] = pd.Categorical(df2['pred_len'], categories=pl_order, ordered=True)

df2.sort_values(by=['data_id', 'model', 'pred_len'], inplace=True)
df2.dropna(inplace=True, thresh=5)
df2

/tmp/ipykernel_9998/111332612.py:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df2_avg = df2.groupby(['model', 'data_id']).mean(numeric_only=True).reset_index()


,model,pred_len,data_id,mse,mae,cov,learning_rate,inner_lr,meta_lr,auxi_loss,rec_lambda,auxi_lambda,reg_lambda,lradj,train_epochs,patience,batch_size,auxi_batch_size,warmup_steps,meta_inner_steps,overlap_ratio,num_tasks,max_norm,first_order,dropout,cycle,meta_type,exp_dir
0,TQNet,96,ETTm1,0.306602,0.348971,0.121591,0.001000,0.001000,0.10000,MSE,1.0,0.0,0.0,type1,30.0,5.0,32.0,64.0,500.0,1.000000,0.0,2.00,5.0,1.0,0.5000,96.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
1,TQNet,192,ETTm1,0.352414,0.376268,0.287309,0.001000,0.001000,0.01000,MSE,1.0,0.0,0.0,type1,30.0,5.0,32.0,64.0,500.0,3.000000,0.0,2.00,5.0,1.0,0.5000,96.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
2,TQNet,336,ETTm1,0.382595,0.397512,0.169736,0.001000,0.001000,0.20000,MSE,1.0,0.0,0.0,type1,30.0,5.0,32.0,64.0,500.0,1.000000,0.0,4.00,5.0,1.0,0.5000,96.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
3,TQNet,720,ETTm1,0.441163,0.434478,0.222437,0.001000,0.001000,0.20000,MSE,1.0,0.0,0.0,type1,30.0,5.0,32.0,64.0,500.0,5.000000,0.0,2.00,5.0,1.0,0.5000,96.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
115,TQNet,Avg,ETTm1,0.370693,0.389307,0.200268,0.001000,0.001000,0.12750,NaN,1.0,0.0,0.0,NaN,30.0,5.0,32.0,64.0,500.0,2.500000,0.0,2.50,5.0,1.0,0.5000,96.0,NaN,NaN
36,PDF,96,ETTm1,0.320094,0.358292,0.162693,0.002000,0.002000,0.05000,MSE,1.0,0.0,0.0,type1,100.0,10.0,32.0,64.0,500.0,1.000000,0.0,3.00,5.0,1.0,0.5000,24.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
37,PDF,192,ETTm1,0.360833,0.379950,0.212361,0.001000,0.001000,0.05000,MSE,1.0,0.0,0.0,type1,100.0,10.0,32.0,64.0,500.0,1.000000,0.0,3.00,5.0,1.0,0.5000,24.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
38,PDF,336,ETTm1,0.390477,0.400542,0.250300,0.002000,0.002000,0.05000,MSE,1.0,0.0,0.0,type1,100.0,10.0,32.0,64.0,500.0,2.000000,0.0,3.00,5.0,1.0,0.5000,24.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
39,PDF,720,ETTm1,0.451229,0.436571,0.310318,0.002000,0.002000,0.05000,MSE,1.0,0.0,0.0,type1,100.0,10.0,32.0,64.0,500.0,2.000000,0.0,3.00,5.0,1.0,0.5000,24.0,all,/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/P...
124,PDF,Avg,ETTm1,0.380658,0.393839,0.233918,0.001750,0.001750,0.05000,NaN,1.0,0.0,0.0,NaN,100.0,10.0,32.0,64.0,500.0,1.500000,0.0,3.00,5.0,1.0,0.5000,24.0,NaN,NaN


In [8]:
save_root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3'
os.makedirs(save_root, exist_ok=True)
dfconfig = pd.read_csv(f"{save_root}/finetune_best.csv")

for model in ['TQNet']:
    for dst in ['ETTh1', 'ETTh2', 'ETTm1', 'ETTm2', 'ECL', 'Weather']:
        print(f"Model: {model}, Data ID: {dst}")
        configs = ['    case $pl in']
        for pl in [96, 192, 336, 720]:
            _df = dfconfig[(dfconfig.model == model) & (dfconfig.data_id == dst) & (dfconfig.pred_len == pl)]
            print(_df.exp_dir.values[0])
            conf = _df[['learning_rate', 'inner_lr', 'meta_lr', 'warmup_steps', 'max_norm', 'num_tasks', 'overlap_ratio', 'meta_inner_steps', 'reg_lambda', 'batch_size', 'auxi_batch_size', 'lradj', 'rec_lambda', 'auxi_loss', 'meta_type']].to_numpy().tolist()[0]
            assert conf[14] == 'all'
            conf_str = conf_str = f"        {pl}) lr={conf[0]} lr_inner={conf[1]} lr_meta={conf[2]} meta_steps={conf[3]} max_norm={conf[4]} num_tasks={conf[5]} overlap_ratio={conf[6]} meta_inner_steps={conf[7]} reg_lambda={conf[8]} batch_size={conf[9]} auxi_batch_size={conf[10]} lradj={conf[11]} lambda={conf[12]} auxi_loss={conf[13]};;"

            configs.append(conf_str)
        configs.append("    esac")
        print('\n'.join(configs))
        print('\n\n')
        
    for dst in ['PEMS03', 'PEMS08']:
        print(f"Model: {model}, Data ID: {dst}")
        configs = ['    case $pl in']
        for pl in [12, 24, 36, 48]:
            _df = dfconfig[(dfconfig.model == model) & (dfconfig.data_id == dst) & (dfconfig.pred_len == pl)]
            print(_df.exp_dir.values[0])
            conf = _df[['learning_rate', 'inner_lr', 'meta_lr', 'warmup_steps', 'max_norm', 'num_tasks', 'overlap_ratio', 'meta_inner_steps', 'reg_lambda', 'batch_size', 'auxi_batch_size', 'lradj', 'rec_lambda', 'auxi_loss', 'meta_type']].to_numpy().tolist()[0]
            assert conf[14] == 'all'
            conf_str = conf_str = f"        {pl}) lr={conf[0]} lr_inner={conf[1]} lr_meta={conf[2]} meta_steps={conf[3]} max_norm={conf[4]} num_tasks={conf[5]} overlap_ratio={conf[6]} meta_inner_steps={conf[7]} reg_lambda={conf[8]} batch_size={conf[9]} auxi_batch_size={conf[10]} lradj={conf[11]} lambda={conf[12]} auxi_loss={conf[13]};;"

            configs.append(conf_str)
        configs.append("    esac")
        print('\n'.join(configs))
        print('\n\n')


Model: TQNet, Data ID: ETTh1
/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/results_ML3/finetune/TQNet_ETTh1_96_1.0_0.0_0.005_0.005_0.2_type1_30_5_32_64_0.0_5.0_3_0.0_3_MSE_24_0.5_linear_1_1_300
/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/results_ML3/finetune/TQNet_ETTh1_192_1.0_0.0_0.005_0.005_0.02_type1_30_5_32_64_0.0_5.0_1_0.0_3_MSE_24_0.5_linear_1_1_300
/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/results_ML3/finetune/TQNet_ETTh1_336_1.0_0.0_0.002_0.002_0.01_type1_30_5_32_64_0.0_5.0_3_0.0_4_MSE_24_0.5_linear_1_1_300
/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/results_ML3/finetune/TQNet_ETTh1_720_1.0_0.0_0.005_0.005_0.01_type1_30_5_32_64_0.0_5.0_5_0.0_2_MSE_24_0.5_linear_1_1_300
    case $pl in
        96) lr=0.005 lr_inner=0.005 lr_meta=0.2 meta_steps=300 max_norm=5.0 num_tasks=3 overlap_ratio=0.0 meta_inner_steps=3 reg_lambda=0.0 batch_size=32 auxi_batch_size=64 lradj=type1 lambda=1.0 auxi_loss=MSE;;
        192) lr=0.005 lr_inner=0.

# report to latex

## load and merge data

In [10]:
save_root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/logs'
dfb = pd.read_csv(f"{save_root}/baselines_chosen.csv")

min_mode = 'each'
columns = ['model', 'data_id', 'learning_rate', 'batch_size', 'patience', 'individual', 'train_epochs']
if min_mode == 'group':
    mse_mean = dfb.groupby(columns)['mse'].mean().reset_index()
    idx = mse_mean.groupby(['model', 'data_id'])['mse'].idxmin()
    best_model = mse_mean.loc[idx]
    dfb = dfb.merge(best_model[columns], on=columns, how='inner')
elif min_mode == 'each':
    min_mse_idx = dfb.groupby(['model', 'data_id', 'pred_len'])['mse'].idxmin()
    dfb = dfb.loc[min_mse_idx]

dfb = dfb[['model', 'pred_len', 'data_id', 'mse', 'mae']]
datasets = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Weather', 'PEMS03', 'PEMS08']
dfb = dfb[dfb['data_id'].isin(datasets)]


save_root = '/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3'
dff = pd.read_csv(f'{save_root}/finetune_best.csv')

dff = dff[
    ((dff.data_id == 'ETTm1') & (dff.model == 'TQNet')) |
    ((dff.data_id == 'ETTm2') & (dff.model == 'TQNet')) |
    ((dff.data_id == 'ETTh1') & (dff.model == 'TQNet')) |
    ((dff.data_id == 'ETTh2') & (dff.model == 'TQNet')) |
    ((dff.data_id == 'ECL') & (dff.model == 'TQNet')) |
    # ((dff.data_id == 'Traffic') & (dff.model == 'TQNet')) |
    ((dff.data_id == 'Weather') & (dff.model == 'TQNet')) |
    ((dff.data_id == 'PEMS03') & (dff.model == 'TQNet')) |
    ((dff.data_id == 'PEMS08') & (dff.model == 'TQNet'))
]

dff['model'] = 'QDF'
dff.to_csv(f"{save_root}/long_term.csv", index=False)
dff = dff[['model', 'pred_len', 'data_id', 'mse', 'mae']]

dft = pd.concat([dfb, dff], axis=0)

# dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Traffic', 'Weather', 'PEMS03', 'PEMS08']
dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Weather', 'PEMS03', 'PEMS08']
dft['data_id'] = pd.Categorical(dft['data_id'], categories=dst_order, ordered=True)

model_order = ['QDF', 'TQNet', 'PDF', 'Fredformer', 'iTransformer', 'FreTS', 'TimesNet', 'MICN', 'TiDE', 'PatchTST', 'DLinear']
dft = dft[dft['model'].isin(model_order)]
dft['model'] = pd.Categorical(dft['model'], categories=model_order, ordered=True)

dft_avg = dft.groupby(['model', 'data_id']).mean(numeric_only=True).reset_index()
dft_avg['pred_len'] = 'Avg'

dft = pd.concat([dft, dft_avg]).reset_index(drop=True)
dft.sort_values(by=['model', 'data_id', 'pred_len'], inplace=True)

dft_show = dft.set_index(['data_id', 'pred_len', 'model']).unstack('model').swaplevel(axis=1)
columns = []
for model in dft_show.columns.levels[0]:
    columns.append((model, 'mse'))
    columns.append((model, 'mae'))
dft_show = dft_show[columns]
dft_show

/tmp/ipykernel_1486806/2195932971.py:49: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dft_avg = dft.groupby(['model', 'data_id']).mean(numeric_only=True).reset_index()


model                  QDF               TQNet                 PDF            \
                       mse       mae       mse       mae       mse       mae   
data_id pred_len                                                               
ETTm1   96        0.306602  0.348971  0.310382  0.351721  0.326234  0.363356   
        192       0.352414  0.376268  0.356100  0.377443  0.364817  0.381136   
        336       0.382595  0.397512  0.387569  0.399900  0.396639  0.402112   
        720       0.441163  0.434478  0.450021  0.436546  0.458370  0.437322   
        Avg       0.370693  0.389307  0.376018  0.391403  0.386515  0.395981   
ETTm2   96        0.170291  0.252600  0.174746  0.255768  0.176168  0.263647   
        192       0.233855  0.293836  0.242704  0.300489  0.245158  0.310199   
        336       0.290130  0.331435  0.297452  0.336435  0.305491  0.345152   
        720       0.386822  0.388532  0.394070  0.393085  0.403959  0.403241   
        Avg       0.270274  0.316601  0.277243  0.321444  0.282694  0.330560   
ETTh1   96        0.365111  0.388897  0.372007  0.391440  0.388068  0.400122   
        192       0.427475  0.421499  0.430498  0.424173  0.440350  0.428146   
        336       0.465918  0.448515  0.486262  0.454420  0.483253  0.448762   
        720       0.466485  0.466534  0.506832  0.485931  0.495008  0.481555   
        Avg       0.431247  0.431361  0.448900  0.438991  0.451670  0.439646   
ETTh2   96        0.285880  0.337922  0.293373  0.342665  0.290784  0.340032   
        192       0.361035  0.388219  0.363695  0.390391  0.374348  0.390743   
        336       0.407637  0.422494  0.411333  0.424371  0.413972  0.425852   
        720       0.419220  0.438823  0.429976  0.443821  0.421089  0.439893   
        Avg       0.368443  0.396864  0.374594  0.400312  0.375048  0.399130   
ECL     96        0.134708  0.228838  0.143456  0.237354  0.175417  0.259347   
        192       0.153000  0.244973  0.160641  0.252081  0.181722  0.265927   
        336       0.169068  0.262404  0.178238  0.269703  0.197201  0.281979   
        720       0.201599  0.290306  0.217629  0.302696  0.237404  0.315331   
        Avg       0.164594  0.256630  0.174991  0.265458  0.197936  0.280646   
Weather 96        0.158300  0.200693  0.159965  0.202723  0.181140  0.221251   
        192       0.206637  0.244936  0.209872  0.247166  0.231525  0.262294   
        336       0.262811  0.286290  0.266696  0.288953  0.285225  0.299993   
        720       0.342250  0.339154  0.346121  0.342470  0.360232  0.348378   
        Avg       0.242499  0.267768  0.245663  0.270328  0.264530  0.282979   
PEMS03  12        0.064309  0.167092  0.097305  0.179638  0.091645  0.204271   
        24        0.080374  0.188680  0.098929  0.204429  0.148696  0.260927   
        36        0.097987  0.208369  0.123181  0.229999  0.210095  0.313705   
        48        0.111981  0.223476  0.157448  0.255610  0.274832  0.364260   
        Avg       0.088663  0.196904  0.119216  0.217419  0.181317  0.285790   
PEMS08  12        0.074487  0.175849  0.078706  0.182539  0.099847  0.208661   
        24        0.104101  0.207774  0.117204  0.222451  0.167697  0.272787   
        36        0.134275  0.236954  0.157844  0.260074  0.244455  0.333303   
        48        0.168359  0.263294  0.203122  0.295390  0.327415  0.388973   
        Avg       0.120305  0.220968  0.139219  0.240114  0.209853  0.300931   

model            Fredformer           iTransformer               FreTS  \
                        mse       mae          mse       mae       mse   
data_id pred_len                                                         
ETTm1   96         0.326369  0.360869     0.337877  0.372283  0.341839   
        192        0.365194  0.382132     0.381666  0.396060  0.384536   
        336        0.395987  0.404369     0.426993  0.423962  0.415986   
        720        0.459217  0.444342     0.495750  0.462558  0.513302   
        Avg        0.386692  0.397928    

## save to latex table

In [11]:
save_root = "/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3"
# dft_show.to_latex(f"{save_root}/long_term.tex")



# dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Traffic', 'Weather', 'PEMS03', 'PEMS08']
dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Weather', 'PEMS03', 'PEMS08']
model_order = ['QDF', 'TQNet', 'PDF', 'Fredformer', 'iTransformer', 'FreTS', 'TimesNet', 'MICN', 'TiDE', 'PatchTST', 'DLinear']
# 初始化统计字典：每个模型的MSE第1次数和MAE第1次数
mse_count = {model: 0 for model in model_order}
mae_count = {model: 0 for model in model_order}


contents = []
stats_contents = []  # 存储统计行内容
for dst in dst_order:
    dfi = dft[dft.data_id == dst]

    dfi['mse_rank'] = dfi.groupby('pred_len')['mse'].rank(ascending=True, method='dense')
    dfi['mae_rank'] = dfi.groupby('pred_len')['mae'].rank(ascending=True, method='dense')

    # MSE第1名：mse_rank=1的模型
    mse_first_models = dfi[dfi['mse_rank'] == 1]['model']
    for model in mse_first_models:
        if model in mse_count:  # 确保模型在model_order中
            mse_count[model] += 1
    
    # MAE第1名：mae_rank=1的模型
    mae_first_models = dfi[dfi['mae_rank'] == 1]['model']
    for model in mae_first_models:
        if model in mae_count:
            mae_count[model] += 1

    for metric in ['mse', 'mae']:
        dfi[f'{metric}_3f'] = dfi[metric].apply(lambda x: "{:.3f}".format(x))
        # 按排名标记：第1名→\bst{}，第2名→\subbst{}，其他→原格式
        conditions = [dfi[f'{metric}_rank'] == 1, dfi[f'{metric}_rank'] == 2]
        choices = [r'\bst{' + dfi[f'{metric}_3f'] + '}', r'\subbst{' + dfi[f'{metric}_3f'] + '}']
        dfi[f'{metric}_processed'] = np.select(conditions, choices, default=dfi[f'{metric}_3f'])
    dfi = dfi[['model', 'pred_len', 'mse_processed', 'mae_processed']]
    dfi.columns = ['model', 'pred_len', 'mse', 'mae']
    dfi['mse'] = dfi['mse'].apply(lambda x: r"\scalea{" + x + "}")  # 应用\scalea
    dfi['mae'] = dfi['mae'].apply(lambda x: r"\scalea{" + x + "}")
    for i, (pl, group) in enumerate(dfi.groupby('pred_len')):
        if i == 0:
            title = r'\multirow{5}{*}{{\rotatebox{90}{\scalebox{0.95}{' + dst + '}}}}'
            contents.append(title)
        line = f"& {pl} "
        for j, row in enumerate(group.itertuples(index=False)):
            line += f"& {row.mse} & {row.mae}"
        line += " \\\\"
        contents.append(line)
        if i == 3:
            contents.append(r"\cmidrule(lr){2-24}")
        elif i  == 4:
            contents.append("\\midrule\n")

stats_title = r"\multicolumn{2}{c|}{\scalea{{$1^{\text{st}}$ Count}}}"  # 标题：合并前两列
stats_values = []
for model in model_order:
    mse = mse_count[model]
    mae = mae_count[model]
    # 次数>0时用\bst{}标记，否则0
    mse_str = r"\scalea{\bst{" + str(mse) + "}}" if mse > 0 else r"\scalea{" + str(mse) + "}"
    mae_str = r"\scalea{\bst{" + str(mae) + "}}" if mae > 0 else r"\scalea{" + str(mae) + "}"
    stats_values.append(f"{mse_str} & {mae_str}")  # 每个模型的MSE/MAE次数
# 合并统计行
stats_line = f"{stats_title} & {' & '.join(stats_values)} \\\\"
stats_contents.append(stats_line)

print(sum(mse_count.values()), sum(mae_count.values()))
print(max(mse_count.values()), max(mae_count.values()))

with open(f"{save_root}/long_term.tex", "w") as f:
    f.write("\n".join(contents + stats_contents))

40 40
39 39


/tmp/ipykernel_1486806/975220537.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['mse_rank'] = dfi.groupby('pred_len')['mse'].rank(ascending=True, method='dense')
/tmp/ipykernel_1486806/975220537.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['mae_rank'] = dfi.groupby('pred_len')['mae'].rank(ascending=True, method='dense')
/tmp/ipykernel_1486806/975220537.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

## write avg

In [13]:
save_root = "/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3"
# dft_show.to_latex(f"{save_root}/long_term.tex")



dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Weather', 'PEMS03', 'PEMS08']
model_order = ['MetaDF', 'TQNet', 'PDF', 'Fredformer', 'iTransformer', 'FreTS', 'TimesNet', 'MICN', 'TiDE', 'PatchTST', 'DLinear']
# 初始化统计字典：每个模型的MSE第1次数和MAE第1次数
mse_count = {model: 0 for model in model_order}
mae_count = {model: 0 for model in model_order}


contents = []
stats_contents = []  # 存储统计行内容
for dst in dst_order:
    dfi = dft[dft.data_id == dst]

    dfi['mse_rank'] = dfi.groupby('pred_len')['mse'].rank(ascending=True, method='dense')
    dfi['mae_rank'] = dfi.groupby('pred_len')['mae'].rank(ascending=True, method='dense')

    # MSE第1名：mse_rank=1的模型
    mse_first_models = dfi[dfi['mse_rank'] == 1]['model']
    for model in mse_first_models:
        if model in mse_count:  # 确保模型在model_order中
            mse_count[model] += 1
    
    # MAE第1名：mae_rank=1的模型
    mae_first_models = dfi[dfi['mae_rank'] == 1]['model']
    for model in mae_first_models:
        if model in mae_count:
            mae_count[model] += 1

    for metric in ['mse', 'mae']:
        dfi[f'{metric}_3f'] = dfi[metric].apply(lambda x: "{:.3f}".format(x))
        # 按排名标记：第1名→\bst{}，第2名→\subbst{}，其他→原格式
        conditions = [dfi[f'{metric}_rank'] == 1, dfi[f'{metric}_rank'] == 2]
        choices = [r'\bst{' + dfi[f'{metric}_3f'] + '}', r'\subbst{' + dfi[f'{metric}_3f'] + '}']
        dfi[f'{metric}_processed'] = np.select(conditions, choices, default=dfi[f'{metric}_3f'])
    dfi = dfi[['model', 'pred_len', 'mse_processed', 'mae_processed']]
    dfi.columns = ['model', 'pred_len', 'mse', 'mae']
    dfi['mse'] = dfi['mse'].apply(lambda x: r"\scalea{" + x + "}")  # 应用\scalea
    dfi['mae'] = dfi['mae'].apply(lambda x: r"\scalea{" + x + "}")
    for i, (pl, group) in enumerate(dfi.groupby('pred_len')):
        if pl != 'Avg':
            continue
        line = r"\multicolumn{2}{l}{\scalea{" + dst + r"}}" + '\n'
        for j, row in enumerate(group.itertuples(index=False)):
            line += f"& {row.mse} & {row.mae}"
        line += " \\\\"
        contents.append(line)
    if dst != dst_order[-1]:
        contents.append("\\midrule")


with open(f"{save_root}/long_term_avg.tex", "w") as f:
    f.write("\n".join(contents))
print("\n".join(contents))

\multicolumn{2}{l}{\scalea{ETTm1}}
& \scalea{\bst{0.371}} & \scalea{\bst{0.389}}& \scalea{\subbst{0.376}} & \scalea{\subbst{0.391}}& \scalea{0.387} & \scalea{0.396}& \scalea{0.387} & \scalea{0.398}& \scalea{0.411} & \scalea{0.414}& \scalea{0.414} & \scalea{0.421}& \scalea{0.438} & \scalea{0.430}& \scalea{0.396} & \scalea{0.421}& \scalea{0.413} & \scalea{0.407}& \scalea{0.389} & \scalea{0.400}& \scalea{0.403} & \scalea{0.407} \\
\midrule
\multicolumn{2}{l}{\scalea{ETTm2}}
& \scalea{\bst{0.270}} & \scalea{\bst{0.317}}& \scalea{\subbst{0.277}} & \scalea{\subbst{0.321}}& \scalea{0.283} & \scalea{0.331}& \scalea{0.280} & \scalea{0.324}& \scalea{0.295} & \scalea{0.336}& \scalea{0.316} & \scalea{0.365}& \scalea{0.302} & \scalea{0.334}& \scalea{0.308} & \scalea{0.364}& \scalea{0.286} & \scalea{0.328}& \scalea{0.303} & \scalea{0.344}& \scalea{0.342} & \scalea{0.392} \\
\midrule
\multicolumn{2}{l}{\scalea{ETTh1}}
& \scalea{\bst{0.431}} & \scalea{\bst{0.431}}& \scalea{0.449} & \scalea{0.439}& \sc

/tmp/ipykernel_253782/3944324592.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['mse_rank'] = dfi.groupby('pred_len')['mse'].rank(ascending=True, method='dense')
/tmp/ipykernel_253782/3944324592.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['mae_rank'] = dfi.groupby('pred_len')['mae'].rank(ascending=True, method='dense')
/tmp/ipykernel_253782/3944324592.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

## write avg

In [22]:
save_root = "/mnt/tidalfs-bdsz01/usr/panlicheng/workspace/ProjDF-Meta/stats_ML3"
# dft_show.to_latex(f"{save_root}/long_term.tex")



dst_order = ['ETTm1', 'ETTm2', 'ETTh1', 'ETTh2', 'ECL', 'Weather', 'PEMS03', 'PEMS08']
model_order = ['MetaDF', 'TQNet', 'PDF', 'Fredformer', 'iTransformer', 'FreTS', 'TimesNet', 'MICN', 'TiDE', 'PatchTST', 'DLinear']
# 初始化统计字典：每个模型的MSE第1次数和MAE第1次数
mse_count = {model: 0 for model in model_order}
mae_count = {model: 0 for model in model_order}


contents = []
stats_contents = []  # 存储统计行内容
for dst in dst_order:
    dfi = dft[dft.data_id == dst]

    dfi['mse_rank'] = dfi.groupby('pred_len')['mse'].rank(ascending=True, method='dense')
    dfi['mae_rank'] = dfi.groupby('pred_len')['mae'].rank(ascending=True, method='dense')

    # MSE第1名：mse_rank=1的模型
    mse_first_models = dfi[dfi['mse_rank'] == 1]['model']
    for model in mse_first_models:
        if model in mse_count:  # 确保模型在model_order中
            mse_count[model] += 1
    
    # MAE第1名：mae_rank=1的模型
    mae_first_models = dfi[dfi['mae_rank'] == 1]['model']
    for model in mae_first_models:
        if model in mae_count:
            mae_count[model] += 1

    for metric in ['mse', 'mae']:
        dfi[f'{metric}_3f'] = dfi[metric].apply(lambda x: "{:.3f}".format(x))
        # 按排名标记：第1名→\bst{}，第2名→\subbst{}，其他→原格式
        conditions = [dfi[f'{metric}_rank'] == 1, dfi[f'{metric}_rank'] == 2]
        choices = [r'\bst{' + dfi[f'{metric}_3f'] + '}', r'\subbst{' + dfi[f'{metric}_3f'] + '}']
        dfi[f'{metric}_processed'] = np.select(conditions, choices, default=dfi[f'{metric}_3f'])
    dfi = dfi[['model', 'pred_len', 'mse_processed', 'mae_processed']]
    dfi.columns = ['model', 'pred_len', 'mse', 'mae']
    dfi['mse'] = dfi['mse'].apply(lambda x: r"\scalea{" + x + "}")  # 应用\scalea
    dfi['mae'] = dfi['mae'].apply(lambda x: r"\scalea{" + x + "}")
    for i, (pl, group) in enumerate(dfi.groupby('pred_len')):
        if pl != 'Avg':
            continue
        line = r"\multicolumn{2}{l}{\scalea{" + dst + r"}}" + '\n'
        for j, row in enumerate(group.itertuples(index=False)):
            line += f"& {row.mse} & {row.mae}"
        line += " \\\\"
        contents.append(line)
    if dst != dst_order[-1]:
        contents.append("\\midrule")


with open(f"{save_root}/long_term_avg.tex", "w") as f:
    f.write("\n".join(contents))
print("\n".join(contents))

\multicolumn{2}{l}{\scalea{ETTm1}}
& \scalea{\bst{0.371}} & \scalea{\bst{0.389}}& \scalea{\subbst{0.376}} & \scalea{\subbst{0.391}}& \scalea{0.387} & \scalea{0.396}& \scalea{0.387} & \scalea{0.398}& \scalea{0.411} & \scalea{0.414}& \scalea{0.414} & \scalea{0.421}& \scalea{0.438} & \scalea{0.430}& \scalea{0.396} & \scalea{0.421}& \scalea{0.413} & \scalea{0.407}& \scalea{0.389} & \scalea{0.400}& \scalea{0.403} & \scalea{0.407} \\
\midrule
\multicolumn{2}{l}{\scalea{ETTm2}}
& \scalea{\bst{0.270}} & \scalea{\bst{0.317}}& \scalea{\subbst{0.277}} & \scalea{\subbst{0.321}}& \scalea{0.283} & \scalea{0.331}& \scalea{0.280} & \scalea{0.324}& \scalea{0.295} & \scalea{0.336}& \scalea{0.316} & \scalea{0.365}& \scalea{0.302} & \scalea{0.334}& \scalea{0.308} & \scalea{0.364}& \scalea{0.286} & \scalea{0.328}& \scalea{0.303} & \scalea{0.344}& \scalea{0.342} & \scalea{0.392} \\
\midrule
\multicolumn{2}{l}{\scalea{ETTh1}}
& \scalea{\bst{0.431}} & \scalea{\bst{0.431}}& \scalea{0.449} & \scalea{0.439}& \sc

/tmp/ipykernel_253782/3944324592.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['mse_rank'] = dfi.groupby('pred_len')['mse'].rank(ascending=True, method='dense')
/tmp/ipykernel_253782/3944324592.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfi['mae_rank'] = dfi.groupby('pred_len')['mae'].rank(ascending=True, method='dense')
/tmp/ipykernel_253782/3944324592.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index